In [3]:
import numpy as np
import general_robotics_toolbox as rox
import general_robotics_toolbox_invkin as rox_invkin
import dofbot_helper as dofbot
import time
import cv2 
import matplotlib.pyplot as plt
from Arm_Lib import Arm_Device #import the module associated with the arm

Arm = Arm_Device() 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-21vg8nme because the default path (/home/jetson/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
def get_q(): 
    speedtime = 100 #time in milliseconds to reach desired joint position
    q = dofbot.readAllActualJointAngles() # read the current position of all joints
    return q
    '''
    while True: #keep executing the indented code until jnum=0
        jnum = dofbot.getJointNumber() #use our defined function to get the joint number
        #if the joint number provided is 0, loop execution ends
        #if the joint number is not 0, we get the angle, move the joint, and read the angle
        if jnum == 0: 
            break
        else:
            ang = getJointAngle(jnum)   #use our defined function to get the joint angle
            moveJoint(jnum,ang,speedtime) #move the desired joint to the given angle
            time.sleep(1) #add a pause to allow time for joints to move
            angActual = readActualJointAngle(jnum) #read the actual position of the desired joint
            print("Actual joint angle:",angActual)
    print("Program has been terminated by user") #let the user know the program is no longer executing
    '''

In [6]:
def Define_Dofbot(): 
    l0 = 0.061
    l1 = 0.0435
    l2 = 0.08285
    l3 = 0.08285
    l4 = 0.07385
    l5 = 0.05457
    
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    DofbotP = np.array([(l0+l1)*ez, np.zeros((3,)), l2*ex, -l3*ez, np.zeros((3,)), -(l4+l5)*ex]).T
    DofbotH = np.array([ez, -ey, -ey, -ey, -ex]).T
    Dofbot_joint_type = np.array([0,0,0,0,0])

    return rox.Robot(DofbotH, DofbotP, Dofbot_joint_type)

In [31]:
def readAllActualJointAngles2():    
    q = np.zeros((6,1)) #set up a 6x1 array placeholder
    for i in range(1,7): #loop through each joint (Note range(1,N) = 1,2,...,N-1)
        #note in Python the array indexing starts at 0 (the reason for i-1 index for q)
        q[i-1] = Arm.Arm_serial_servo_read(i) #store read angle into corresponding index of q
    return q

In [32]:
def move_joints(q, speedtime=100):
    
    for jnum in range(1, q.shape[0]+1):
        dofbot.moveJoint(jnum, q[jnum-1], speedtime)
        time.sleep(0.05)
    
    tol = np.array([0.02, 0.02, 0.02, 0.001, 0.001, 0.001])
    Nmax = 200
    epsilon = 0.1 
    alpha = 0.1
    robot = Define_Dofbot()
    transform = rox.fwdkin(robot, q)
    J = rox.robotjacobian(robot, q0)
    R0T = transform.R
    P0T = transform.p
    q = ik_Jinverse(robot,q0,R0T,P0T,Nmax,alpha,tol,J)*(180/np.pi)
        
    #moveJoint(jnum,ang,speedtime)

In [7]:
def get_position(robot):
    # 1) Get the theta values
    q = (np.array(get_q())*(np.pi/180))
    
    # 2) Get R0T and P0T from fwdkin 
    transform = rox.fwdkin(robot, q)
    P0T = transform.p 
    R0T = transform.R
    
    return q, P0T, R0T
    

In [8]:
def move_to_desired(robot, desired): 
    
    # 1) Get current position
    q_cur, P_cur, R_cur = get_position(robot)
    
    # 2) Get angles from desired transform location 
    # Double check units that invkin returns (rad or degrees)
    theta = rox_invkin.ur_invkin(robot, desired, q_cur)*(180/np.pi)
    
    # 3) Move robot to the desired angles 
    if len(theta) > 0: 
        move_joints(theta)
    
    return theta
    

In [9]:
def get_transform(q): 
    q = np.array(get_q())*(np.pi/180)
    transform = rox.fwdkin(robot, q)
    return transform

In [10]:
def straight_line(q0, qf, step_size):
    lam = 0
    for i in range(0, int(1/step_size)):
        q_cur = q0*(1-lam)+qf*lam
        move_joints(q_cur)
        lam += step_size

In [11]:
def readAllActualJointAngles():
    q = np.array([Arm.Arm_serial_servo_read(1),Arm.Arm_serial_servo_read(2),Arm.Arm_serial_servo_read(3),Arm.Arm_serial_servo_read(4),Arm.Arm_serial_servo_read(5),Arm.Arm_serial_servo_read(6)])
    return q

In [30]:
def make_board(robot):
    q1 = np.array([85, 64, 2, 90, 90, 166])
    q1_safe = np.array([90, 70, 2, 90, 90, 166])
    q2 = np.array([86, 0, 90, 90, 90, 166])
    q2_safe = np.array([87, 5, 90, 90, 90, 166])
    q3 = np.array([95, 64, 2, 90, 90, 166])
    q3_safe = np.array([93, 70, 2, 90, 90, 166])
    q4 = np.array([94, 0, 90, 90, 90, 166])
    q4_safe = np.array([95, 5, 90, 90, 90, 166])
    q5 = np.array([102, 55, 13, 90, 90, 166])
    q5_safe = np.array([102, 60, 30, 90, 90, 166])
    q6 = np.array([78, 55, 13, 90, 90, 166])
    q6_safe = np.array([78, 60, 13, 90, 90, 166])
    q7 = np.array([80, 30, 48, 90, 90, 166])
    q7_safe = np.array([80, 35, 48, 90, 90, 166])
    q8 = np.array([100, 30, 48, 90, 90, 166])
    q8_safe = np.array([90, 90, 2, 90, 90, 160])

    #board_points = np.array([get_transform(q1), get_transform(q2), get_transform(q3), get_transform(q4), 
    #                         get_transform(q5), get_transform(q6), get_transform(q7), get_transform(q8)])
    # T = rox.Transform(R, P)
    
    # Initialize drawing tic-tac-toe board
    move_joints(q1_safe, 2000)
    time.sleep(3)
    print('In safe position')

    # Draw line 1 to 2
    move_joints(q1, 500)
    print('Drawing line 1 to 2')
    straight_line(q1, q2, 0.03)
    print('Line 1 to 2 drawn')
    move_joints(q2_safe, 2000)
    time.sleep(3)
    move_joints(q3_safe, 2000)
    time.sleep(3)
    print('In safe position')
    
    # Draw line 3 to 4
    move_joints(q3, 500)
    print('Drawing line 3 to 4')
    straight_line(q3, q4, 0.03)
    print('Line 3 to 4 drawn')
    move_joints(q4_safe, 2000)
    time.sleep(3)
    move_joints(q5_safe, 2000)
    time.sleep(3)
    print('In safe positio')
    
    # Draw line 5 to 6
    move_joints(q5, 500)
    print('Drawing line 5 to 6')
    straight_line(q5, q6, 0.03)
    print('Line 5 to 6 drawn')
    move_joints(q6_safe, 2000)
    time.sleep(3)
    move_joints(q7_safe, 2000)
    time.sleep(3)
    print('In safe position')
    
    # Draw line 7 to 8
    move_joints(q7, 500)
    print('Drawing line 7 to 8')
    straight_line(q7, q8, 0.03)
    print('Line 7 to 8 drawn')
    desired = q8
    actual = readAllActualJointAngles()
    
    print("Error for board is:", np.linalg.norm(actual-desired))
    
    move_joints(q8_safe)
    
    print('Tac-tac-toe board is done!')
   

In [98]:
def get_actual():
    actual = np.zeros((6,))
    for i in range(1,7):
        count = 0
        while count == 0 or actual[i-1] is None:
            actual[i-1] = Arm.Arm_serial_servo_read(i)
            count += 1
    return actual

In [100]:
def straight_line_feedback(q0, qf, step_size, k):
    error_list = [] 
    lam = 0
    actual = get_actual()
    error = q0-actual
    error_list.append(np.linalg.norm(error))
    for i in range(0, int(1/step_size)):
        q_cur = q0*(1-lam) + qf*lam + k*error 
        q_cur = np.where(q_cur > 175, 175, q_cur)
        q_cur = np.where(q_cur < 0, 0, q_cur)
        
        if(np.isnan(q_cur).any()): 
            q_cur = q0*(1-lam) + qf*lam
        
        move_joints(q_cur)
        actual = get_actual()
        error = q_cur - actual
        error_list.append(np.linalg.norm(error))
        
        lam += step_size
    return np.array(error_list)

In [ ]:
def make_board_feedback(robot):
    k = 0.3
    q1 = np.array([85, 64, 2, 90, 90, 166])
    q1_safe = np.array([90, 70, 2, 90, 90, 166])
    q2 = np.array([86, 0, 90, 90, 90, 166])
    q2_safe = np.array([87, 5, 90, 90, 90, 166])
    q3 = np.array([95, 64, 2, 90, 90, 166])
    q3_safe = np.array([93, 70, 2, 90, 90, 166])
    q4 = np.array([94, 0, 90, 90, 90, 166])
    q4_safe = np.array([95, 5, 90, 90, 90, 166])
    q5 = np.array([102, 55, 13, 90, 90, 166])
    q5_safe = np.array([102, 60, 30, 90, 90, 166])
    q6 = np.array([78, 55, 13, 90, 90, 166])
    q6_safe = np.array([78, 60, 13, 90, 90, 166])
    q7 = np.array([80, 30, 48, 90, 90, 166])
    q7_safe = np.array([80, 35, 48, 90, 90, 166])
    q8 = np.array([100, 30, 48, 90, 90, 166])
    q8_safe = np.array([90, 90, 2, 90, 90, 160])

    #board_points = np.array([get_transform(q1), get_transform(q2), get_transform(q3), get_transform(q4), 
    #                         get_transform(q5), get_transform(q6), get_transform(q7), get_transform(q8)])
    # T = rox.Transform(R, P)
    
    # Initialize drawing tic-tac-toe board
    move_joints(q1_safe, 2000)
    time.sleep(3)
    print('In safe position')

    # Draw line 1 to 2
    move_joints(q1, 500)
    print('Drawing line 1 to 2')
    error_list = straight_line_feedback(q1, q2, 0.03, k)
    print('Line 1 to 2 drawn')
    move_joints(q2_safe, 2000)
    time.sleep(3)
    move_joints(q3_safe, 2000)
    time.sleep(3)
    print('In safe position')
    
    # Draw line 3 to 4
    move_joints(q3, 500)
    print('Drawing line 3 to 4')
    error_list = straight_line_feedback(q3, q4, 0.03, k)
    print('Line 3 to 4 drawn')
    move_joints(q4_safe, 2000)
    time.sleep(3)
    move_joints(q5_safe, 2000)
    time.sleep(3)
    print('In safe positio')
    
    # Draw line 5 to 6
    move_joints(q5, 500)
    print('Drawing line 5 to 6')
    error_list = straight_line_feedback(q5, q6, 0.03, k)
    print('Line 5 to 6 drawn')
    move_joints(q6_safe, 2000)
    time.sleep(3)
    move_joints(q7_safe, 2000)
    time.sleep(3)
    print('In safe position')
    
    # Draw line 7 to 8
    move_joints(q7, 500)
    print('Drawing line 7 to 8')
    error_list = straight_line_feedback(q7, q8, 0.03, k)
    print('Line 7 to 8 drawn')
    desired = q8
    actual = readAllActualJointAngles()
    
    print("Error for board is:", np.linalg.norm(actual-desired))
    
    move_joints(q8_safe)
    
    print('Tac-tac-toe board is done!')
    return error_list

robot = Define_Dofbot() 
error_list = make_board_feedback(robot)
plt.plot(np.arange(0, len(error_list)), error_list)
plt.title("Error Values Over Time During Proportional Control")
plt.xlabel('Time (in seconds)')
plt.ylabel('Error value')
plt.show()

In [33]:
q_initial = np.array([90, 90, 2, 90, 90, 145])
move_joints(q_initial, 500)
print()

In [12]:
def ik_Jinverse(robot,q0,Rd,Pd,Nmax,alpha,tol,J):
    
    n = (q0).shape[0]
    T = rox.fwdkin(robot,q0)
    R = T.R
    P = T.p
    
    q = np.zeros((n, Nmax+1))
    q[:,0] = q0 # output joint displacements
    p0T = np.zeros((3,Nmax+1)) # output p
    RPY0T = np.zeros((3,Nmax+1)) # output Euler angles
    
    iternum = 0
    
    # get the pose error
    dR = np.matmul(R, Rd.T)
    dX = np.concatenate((np.array(rox.R2rpy(dR))[None].T, np.reshape(P-Pd, (3,1))))
    
    # iterative update    
    while (dX>tol).any():
        if iternum <= Nmax:
            # forward kinematics
            p0T[:,iternum] = rox.fwdkin(robot, q[:, iternum]).p
            R = rox.fwdkin(robot, q[:, iternum]).R
            
            Jq = rox.robotjacobian(robot, q[:, iternum])
            RPY0T[:, iternum] = np.reshape(np.array(rox.R2rpy(R))[None].T, (3,))
            
            # get the pose error
            dR = np.matmul(R,Rd.T)
            dX = np.concatenate((np.array(rox.R2rpy(dR))[None].T, np.reshape(p0T[:, iternum]-Pd, (3,1))))
            
            # Jacobian update       
            # dX = [beta s; P-Pd]
            np.copyto(q[:,iternum+1],q[:,iternum]-alpha*np.reshape(np.matmul(np.linalg.pinv(Jq), dX), (n,)))
            iternum = iternum + 1
            
        else:
            break
    
    iternum = iternum - 1
    
    q, p0T, RPY0T = q[:, :iternum], p0T[:, :iternum], RPY0T[:, :iternum]
    
    return q

In [13]:
def readAllActualJointAngles2():    
    q = np.zeros((6,1)) #set up a 6x1 array placeholder
    for i in range(1,7): #loop through each joint (Note range(1,N) = 1,2,...,N-1)
        #note in Python the array indexing starts at 0 (the reason for i-1 index for q)
        q[i-1] = Arm.Arm_serial_servo_read(i) #store read angle into corresponding index of q
    return q

In [30]:
def straight_line_inv(robot, P0, R0, Pd, q_cur, step_size):
    print("HELLO")
    print(P0)
    print(R0)
    
    #R0 = np.array([[-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
    #P0 = np.array([0.2058, 0.1188, 0.1464])
    q_cur = np.array([25, 50, 75, 30, 30])*(np.pi/180)
    
    tol = np.array([0.02, 0.02, 0.02, 0.001, 0.001, 0.001])
    Nmax = 200
    epsilon = 0.1 
    alpha = 0.1
    
    J = rox.robotjacobian(robot, q_cur)
    q = ik_Jinverse(robot,q_cur,R0,P0,Nmax,alpha,tol,J)
    print(np.rad2deg(q[:, -1]))
    '''
    #theta = rox_invkin.ur_invkin(robot, desired, q_cur)*(180/np.pi)
    print("ENTER")
    lam = 0
    for i in range(0, int(1/step_size)):
        P_cur = P0*(1-lam)+Pd*lam
        desired = rox.Transform(R0, P_cur)
        q_cur = rox_invkin.ur_invkin(robot, desired, q_cur)
        print(q_cur*(180/np.pi))
        lam += step_size
    '''
tol = np.array([0.02, 0.02, 0.02, 0.001, 0.001, 0.001])
Nmax = 200
epsilon = 0.1 
alpha = 0.1
robot = Define_Dofbot()
q_initial = np.array([90, 90, 2, 90, 90, 145])
move_joints(q_initial, 500)
transform = rox.fwdkin(robot, q_initial)
q0 = np.array([90, 80, 2, 80, 90])*(np.pi/180)
q0 = np.array([30, 50, 75, 30, 30])
move_joints(q0, 500)

R0T = np.array([[-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
P0T = np.array([0.2058, 0.1188, 0.1464])
q0 = np.array([25, 50, 75, 30, 30])*(np.pi/180)
J = rox.robotjacobian(robot, q0)
q = ik_Jinverse(robot,q0,R0T,P0T,Nmax,alpha,tol,J)*(180/np.pi)
q = q[:, -1]

q0 = np.array([30, 45, 75, 25, 30])
move_joints(q0, 500)
actual =readAllActualJointAngles2()[:5]


print(q)
print(actual)
print("IK Error:", np.linalg.norm(actual-q))

[29.57085253 45.60822546 79.23505823 25.48396132 39.24878898]
[[33.]
 [56.]
 [64.]
 [44.]
 [45.]]
IK Error: 111.60692423495662


In [31]:
robot = Define_Dofbot() 

# Robot goes to initial position
q_initial = np.array([90, 90, 2, 90, 90, 145])
move_joints(q_initial, 500)
print('Begin drawing tic-tac-toe board')

# Wait 20 seconds
time.sleep(20)

q_initial = np.array([90, 90, 2, 90, 90, 166])
move_joints(q_initial, 500)

# Draw the tic-ta-toe board
make_board(robot)

# Robot returns to initial position 

Begin drawing tic-tac-toe board
In safe position
Drawing line 1 to 2
Line 1 to 2 drawn
In safe position
Drawing line 3 to 4
Line 3 to 4 drawn
In safe positio
Drawing line 5 to 6
Line 5 to 6 drawn
In safe position
Drawing line 7 to 8
Line 7 to 8 drawn
Error for board is: 6.4031242374328485
Tac-tac-toe board is done!


In [72]:
q_initial = np.array([90, 90, 2, 90, 90, 145])
move_joints(q_initial, 500)
print()

In [144]:
# Draw X
# X point rotations
qx1 = np.array([87, 22, 60, 90, 90, 166]) # Bottom right
qx4 = np.array([87, 5, 85, 90, 90, 166]) # Top Right
qx2 = np.array([93, 5, 85, 90, 90, 166]) # Top Left
qx3 = np.array([93, 22, 60, 90, 90, 166]) # Bottom Left

# X point safe positions
qx1_safe = np.array([87, 38, 60, 90, 90, 166]) # Bottom right
qx4_safe = np.array([87, 22, 75, 90, 90, 166]) # Top Right
qx2_safe = np.array([93, 21, 75, 90, 90, 166]) # Top Left
qx3_safe = np.array([93, 42, 48, 90, 90, 166]) # Bottom Left

q_initial = np.array([90, 90, 2, 90, 90, 166])
move_joints(q_initial, 500)

# Draw first line of the X
move_joints(qx1_safe, 2000)
move_joints(qx1, 3000)
straight_line(qx1, qx2, 0.03)
move_joints(qx2_safe)

# Draw second line of the X
move_joints(qx3_safe, 2000)
move_joints(qx3, 1000)
straight_line(qx3, qx4, 0.03)
move_joints(qx4_safe)

# Drawing X complete
move_joints(q_initial, 500)


'''
desired = rox.Transform(R0T, P0T)
print(R0T)
print(Pd)
q_cur = rox_invkin.ur_invkin(robot, desired)
'''


'\ndesired = rox.Transform(R0T, P0T)\nprint(R0T)\nprint(Pd)\nq_cur = rox_invkin.ur_invkin(robot, desired)\n'

In [34]:
# Draw 0
# O point rotations
qo1 = np.array([83, 14, 70, 90, 90, 166]) 
qo2 = np.array([81, 4, 85, 90, 90, 166]) 
qo3 = np.array([79, 14, 72, 90, 90, 166]) 
qo4 = np.array([81, 25, 58, 90, 90, 166]) 
qo5 = np.array([84, 14, 70, 90, 90, 166]) 

# Safe positions
qo1_safe = np.array([83, 16, 70, 90, 90, 166])

q_initial = np.array([90, 90, 2, 90, 90, 166])
move_joints(q_initial, 500)

# Draw first line of the O
move_joints(qo1_safe, 2000)
time.sleep(3)
move_joints(qo1, 3000)
straight_line(qo1, qo2, 0.03)
straight_line(qo2, qo3, 0.03)
straight_line(qo3, qo4, 0.03)
straight_line(qo4, qo5, 0.03)
time.sleep(1)

desired = qo5
actual = readAllActualJointAngles()
# Drawing O complete
move_joints(q_initial, 500)
print("Error for O is:", np.linalg.norm(actual-desired))



Error for O is: 6.48074069840786


In [35]:
def drawX(position):
    qx1 = np.array([90, 90, 2, 90, 90, 145])
    qx2 = np.array([90, 90, 2, 90, 90, 145])
    qx3 = np.array([90, 90, 2, 90, 90, 145])
    qx4 = np.array([90, 90, 2, 90, 90, 145])
    
    # Box 1: X point rotations
    if position == 1:
        qx1 = np.array([97, 65, 0, 90, 90, 166]) # Bottom right
        qx2 = np.array([100, 58, 10, 90, 90, 166]) # Top Left
        qx3 = np.array([100, 65, 0, 90, 90, 166]) # Bottom Left
        qx4 = np.array([97, 58, 10, 90, 90, 166]) # Top Right
    
    # Box 2: X point rotations
    elif position == 2:
        qx1 = np.array([86, 65, 0, 90, 90, 166]) # Bottom right
        qx2 = np.array([94, 58, 10, 90, 90, 166]) # Top Left
        qx3 = np.array([94, 65, 0, 90, 90, 166]) # Bottom Left
        qx4 = np.array([86, 58, 10, 90, 90, 166]) # Top Right

    # Box 3: X point rotations
    elif position == 3:
        qx1 = np.array([80, 65, 0, 90, 90, 166]) # Bottom right
        qx2 = np.array([83, 58, 10, 90, 90, 166]) # Top Left
        qx3 = np.array([83, 65, 0, 90, 90, 166]) # Bottom Left
        qx4 = np.array([80, 58, 10, 90, 90, 166]) # Top Right

    # Box 4: X point rotations
    elif position == 4:
        qx1 = np.array([97, 50, 20, 90, 90, 166]) # Bottom right
        qx2 = np.array([100, 40, 35, 90, 90, 166]) # Top Left
        qx3 = np.array([100, 50, 20, 90, 90, 166]) # Bottom Left
        qx4 = np.array([97, 40, 35, 90, 90, 166]) # Top Right

    # Box 5: X point rotations
    elif position == 5:
        qx1 = np.array([86, 50, 20, 90, 90, 166]) # Bottom right
        qx2 = np.array([94, 40, 35, 90, 90, 166]) # Top Left
        qx3 = np.array([94, 50, 20, 90, 90, 166]) # Bottom Left
        qx4 = np.array([86, 40, 35, 90, 90, 166]) # Top Right

    # Box 6: X point rotations
    elif position == 6:
        qx1 = np.array([80, 50, 20, 90, 90, 166]) # Bottom right
        qx2 = np.array([83, 40, 35, 90, 90, 166]) # Top Left
        qx3 = np.array([83, 50, 20, 90, 90, 166]) # Bottom Left
        qx4 = np.array([80, 40, 35, 90, 90, 166]) # Top Right

    # Box 7: X point rotations
    elif position == 7:
        qx1 = np.array([97, 22, 60, 90, 90, 166]) # Bottom right
        qx2 = np.array([100, 3, 85, 90, 90, 166]) # Top Left
        qx3 = np.array([100, 22, 60, 90, 90, 166]) # Bottom Left
        qx4 = np.array([97, 3, 85, 90, 90, 166]) # Top Right

    # Box 8: X point rotations
    elif position == 8:
        qx1 = np.array([87, 22, 60, 90, 90, 166]) # Bottom right
        qx2 = np.array([93, 3, 85, 90, 90, 166]) # Top Left
        qx3 = np.array([93, 22, 60, 90, 90, 166]) # Bottom Left
        qx4 = np.array([87, 3, 85, 90, 90, 166]) # Top Right

    # Box 9: X point rotations
    elif position == 9:
        qx1 = np.array([80, 22, 60, 90, 90, 166]) # Bottom right
        qx2 = np.array([83, 3, 85, 90, 90, 166]) # Top Left
        qx3 = np.array([83, 22, 60, 90, 90, 166]) # Bottom Left
        qx4 = np.array([80, 3, 85, 90, 90, 166]) # Top Right
    
    safe = np.array([0,5,0,0,0,0])
    
    q_initial = np.array([90, 90, 2, 90, 90, 166])
    move_joints(q_initial, 500)

    # Draw first line of the X
    move_joints(qx1+safe, 2000)
    time.sleep(3)
    move_joints(qx1, 3000)
    straight_line(qx1, qx2, 0.03)
    move_joints(qx2+safe)

    # Draw second line of the X
    move_joints(qx3+safe, 2000)
    time.sleep(3)
    move_joints(qx3, 1000)
    straight_line(qx3, qx4, 0.03)
    
    desired = qx4
    actual = readAllActualJointAngles()
    move_joints(qx4+safe)

    # Drawing X complete
    move_joints(q_initial, 500)
    
    print("POSITION IS", position)
    print("Error for X is:", np.linalg.norm(actual-desired))
    
drawX(8)

POSITION IS 8
Error for X is: 6.48074069840786


In [ ]:
####	TIC TAC TOE    ####

#START;


#FUNCTIONS;

def default():
	#To be printed as Default;
    print("\nWelcome! Let's play TIC TAC TOE!\n")


def rules():
    print("The board will look like this!")
    print("The positions of this 3 x 3 board is same as the right side of your key board.\n")
    print(" 7 | 8 | 9 ")
    print("-----------")
    print(" 4 | 5 | 6 ")
    print("-----------")
    print(" 1 | 2 | 3 ")
    print("\nYou just have to input the position(1-9).")


def play():
	#Asking if the player is ready;
    return input("\nAre you ready to play the game? Enter [Y]es or [N]o.\t").upper().startswith('Y')


def names():
    #Player names input;
    
    p1_name=input("\nEnter NAME of PLAYER 1:\t").capitalize()
    p2_name=input("Enter NAME of PLAYER 2:\t").capitalize()
    return (p1_name, p2_name)


def choice():
    #Player choice input;
    p1_choice = ' '
    p2_choice = ' '
    while p1_choice != 'X' or p1_choice != 'O':          #while loop; if the entered value isn't X or O;
        
        #WHILE LOOP STARTS

        p1_choice = input(f"\n{p1_name}, Do you want to be X or O?\t")[0].upper()
        #The input above has [0].upper() in the end;
        #So the user can enter x, X, xxxx or XXX; the input will always be taken as X;
        #Thereby, increasing the user input window;

        if p1_choice == 'X' or p1_choice == 'O':
            #if entered value is X or O; get out of the loop; 
            break
        print("INVALID INPUT! Please Try Again!") 
        #if the entered value isn't X or O, re-run the while loop;

        #WHILE LOOP ENDS
    #Assigning the value to p2 and then diplaying the values;
    if p1_choice == 'X':
        p2_choice = 'O'
    elif p1_choice == 'O':
        p2_choice = 'X'
    
    return (p1_choice, p2_choice)



def first_player():
    #This function will randomly decide who will go first;
    import random
    return random.choice((0, 1))


def display_board(board, avail):
    print("    " + " {} | {} | {} ".format(board[7],board[8],board[9]) + "            " + " {} | {} | {} ".format(avail[7],avail[8],avail[9]))
    print("    " + "-----------" + "            " + "-----------")
    print("    " + " {} | {} | {} ".format(board[4],board[5],board[6]) + "            " + " {} | {} | {} ".format(avail[4],avail[5],avail[6]))
    print("    " + "-----------" + "            " + "-----------")
    print("    " + " {} | {} | {} ".format(board[1],board[2],board[3]) + "            " + " {} | {} | {} ".format(avail[1],avail[2],avail[3]))


def player_choice(board, name, choice):
    position = 0
    #Initialising position as 0^; so it passes through the while loop; 
    while position not in [1,2,3,4,5,6,7,8,9] or not space_check(board, position):
        position = int(input(f'\n{name} ({choice}), Choose your next position: (1-9) \t'))
        
        if position not in [1,2,3,4,5,6,7,8,9] or not space_check(board, position) or position == "": 
            #To check whether the given position is in the set [1-9] or whether it is empty or occupied;
            print(f"INVALID INPUT. Please Try Again!\n")   
    print("\n")        
    return position


# THIS IS THEFUNCTION WHERE AI IS ADDED:
def CompAI(board, name, choice):
    position = 0
    possibilities = [x for x, letter in enumerate(board) if letter == ' ' and x != 0]
    
    # including both X and O, since if computer will win, he will place a choice there, but if the component will win --> we have to block that move
    for let in ['O', 'X']:
        for i in possibilities:
            # Creating a copy of the board everytime, placing the move and checking if it wins;
            # Creating a copy like this  and not this boardCopy = board, since changes to boardCopy changes the original board;
            boardCopy = board[:]
            boardCopy[i] = let
            if(win_check(boardCopy, let)):
                position = i
                return position

    openCorners = [x for x in possibilities if x in [1, 3, 7, 9]]
    
    if len(openCorners) > 0:
        position = selectRandom(openCorners)
        return position

    if 5 in possibilities:
        position = 5
        return position

    openEdges = [x for x in possibilities if x in [2, 4, 6, 8]]
    
    if len(openEdges) > 0:
        position = selectRandom(openEdges)
        return position



def selectRandom(board):
    import random
    ln = len(board)
    r = random.randrange(0,ln)
    return board[r]


def place_marker(board, avail, choice, position):
    #To mark/replace the position on the board list;
    board[position] = choice
    avail[position] = ' '


def space_check(board, position):
    #To check whether the given position is empty or occupied;
    return board[position] == ' '


def full_board_check(board):
    #To check if the board is full, then the game is a draw;
    for i in range(1,10):
        if space_check(board, i):
            return False
    return True


def win_check(board, choice):
    #To check if one of the following patterns are true; then the respective player has won!;
    
    #HORIZONTAL CHECK;
    return ( 
       ( board[1] == choice and board[2] == choice and board[3] == choice )
    or ( board[4] == choice and board[5] == choice and board[6] == choice )
    or ( board[7] == choice and board[8] == choice and board[9] == choice )
    #VERTICAL CHECK;
    or ( board[1] == choice and board[4] == choice and board[7] == choice )
    or ( board[2] == choice and board[5] == choice and board[8] == choice )
    or ( board[3] == choice and board[6] == choice and board[9] == choice )
    #DIAGONAL CHECK;
    or ( board[1] == choice and board[5] == choice and board[9] == choice )
    or ( board[3] == choice and board[5] == choice and board[7] == choice )  )

def delay(mode):
    if mode == 2:
        import time
        time.sleep(2)

def replay():
    #If the users want to play the game again?
    return input('\nDo you want to play again? Enter [Y]es or [N]o: ').lower().startswith('y')





#MAIN PROGRAM STARTS;

input("Press ENTER to start!")

default()
rules()


while True:
    ####################################################################################
    
    #Creating the board as a list; to be kept replacing it with user input;
    theBoard = [' ']*10
    
    #Creating the available options on the board:
    available = [str(num) for num in range(0,10)] # a List Comprehension
    #available = '0123456789'
    
    #print("\n[0]. Player vs. Computer")
    #print("[1]. Player vs. Player")
    #print("[2]. Computer vs. Computer")
    #mode = int(input("\nSelect an option [0]-[2]: "))
    mode = 0
    if mode == 1:
        #Asking Names;
        p1_name, p2_name = names()
        # Asking Choices; Printing choices; X or O;
        p1_choice, p2_choice = choice()
        print(f"\n{p1_name}:", p1_choice)
        print(f"{p2_name}:", p2_choice)

    elif mode == 0:
        p1_name = input("\nEnter NAME of PLAYER who will go against the Computer:\t").capitalize()
        p2_name = "Computer"
        # Asking Choices; Printing choices; X or O;
        p1_choice, p2_choice = choice()
        print(f"\n{p1_name}:", p1_choice)
        print(f"{p2_name}:", p2_choice)
    
    else:
        p1_name = "Computer1"
        p2_name = "Computer2"
        p1_choice, p2_choice = "X", "O"
        print(f"\n{p1_name}:", p1_choice)
        print(f"\n{p2_name}:", p2_choice)


    
    #Printing randomly who will go first;
    if first_player():
        turn = p2_name
    else:
        turn = p1_name

    print(f"\n{turn} will go first!")
    
    #Asking the user, if ready to play the game; Output will be True or False;
    if(mode == 2):
        ent = input("\nThis is going to be fast! Press Enter for the battle to begin!\n")
        play_game = 1
    else:
        play_game = play()   
    
    while play_game:
        
        ############################
        #PLAYER1
        if turn == p1_name:
            
            #Displaying the board;
            display_board(theBoard, available)

            #Position of the input;
            if mode != 2:
                position = player_choice(theBoard, p1_name, p1_choice)
            else:
                position = CompAI(theBoard, p1_name, p1_choice)
                print(f'\n{p1_name} ({p1_choice}) has placed on {position}\n')
            
            #Replacing the ' ' at *position* to *p1_choice* in *theBoard* list; 
            place_marker(theBoard, available, p1_choice, position)
            
            #To check if Player 1 has won after the current input;
            if win_check(theBoard, p1_choice):
                display_board(theBoard, available)
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                if(mode):
                    print(f'\n\nCONGRATULATIONS {p1_name}! YOU HAVE WON THE GAME!\n\n')
                else:
                    print('\n\nTHE Computer HAS WON THE GAME!\n\n')
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                play_game = False
                
            else:
                #To check if the board is full; if yes, the game is a draw;
                if full_board_check(theBoard):
                    display_board(theBoard, available)
                    print("~~~~~~~~~~~~~~~~~~")
                    print('\nThe game is a DRAW!\n')
                    print("~~~~~~~~~~~~~~~~~~")
                    break
                #If none of the above is possible, next turn of Player 2;    
                else:
                    turn = p2_name
                    
                    
        ############################
        #PLAYER2   
        # IF IT IS THE ROBOTS TURN PLACE THE X or O on the board          
        elif turn == p2_name:
            
            #Displaying the board;
            display_board(theBoard, available)

            #Position of the input;
            if(mode == 1):
                position = player_choice(theBoard, p2_name, p2_choice)
            else:
                position = CompAI(theBoard, p2_name, p2_choice)
                #drawX(position) UNCOMMENT THIS TO DRAW THE TIC-TAC-TOE
                print(f'\n{p2_name} ({p2_choice}) has placed on {position}\n')
            
            #Replacing the ' ' at *position* to *p2_choice* in *theBoard* list; 
            place_marker(theBoard, available, p2_choice, position)
            
            #To check if Player 2 has won after the current input;
            if win_check(theBoard, p2_choice):
                display_board(theBoard, available)
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                if(mode):
                    print(f'\n\nCONGRATULATIONS {p2_name}! YOU HAVE WON THE GAME!\n\n')
                else:
                    print('\n\nTHE {p1_name} HAS WON THE GAME!\n\n')
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                play_game = False
                
            else:
                #To check if the board is full; if yes, the game is a draw;
                if full_board_check(theBoard):
                    display_board(theBoard, available)
                    print("~~~~~~~~~~~~~~~~~~")
                    print('\nThe game is a DRAW!\n')
                    print("~~~~~~~~~~~~~~~~~~")
                    break
                #If none of the above is possible, next turn of Player 2;       
                else:
                    turn = p1_name
        
                    
    #If the users want to play the game again?                
    break
        
    ####################################################################################
    
print("\n\n\t\t\tTHE END!")



#END    